# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import qgrid
from flask import jsonify
import secret

In [2]:
# connect to local database 
from sqlalchemy import create_engine
# {secret.user_pass_post}
engine = create_engine(f"postgresql://{secret.user_pass_post}@localhost:5432/medicareDB")
engine.table_names()

['physicians_and_supplier2012a',
 'physicians_and_supplier2012b',
 'home_health_agencies_2013',
 'home_health_agencies_2014',
 'hospice_providers_2014',
 'hospital_general_info',
 'inpatient_charges_2011',
 'inpatient_charges_2012',
 'inpatient_charges_2013',
 'inpatient_charges_2014',
 'inpatient_charges_2015',
 'nursing_facilities_2013',
 'nursing_facilities_2014',
 'outpatient_charges_2011',
 'outpatient_charges_2012',
 'outpatient_charges_2013',
 'outpatient_charges_2014',
 'outpatient_charges_2015',
 'physicians_and_supplier2013b',
 'physicians_and_supplier2013a',
 'physicians_and_supplier2014a',
 'physicians_and_supplier2014b',
 'physicians_and_supplier2015a',
 'physicians_and_supplier2015b',
 'part_d_prescriber_2014a',
 'part_d_prescriber_2014b',
 'part_d_prescriber_2014c',
 'part_d_prescriber_2014d',
 'part_d_prescriber_2014_table',
 'data/CLEAN_part_d_FINAL',
 'CLEAN_in_out_patients_charges_metadata',
 'CLEAN_outpatients_charges_combined',
 'CLEAN_inpatients_charges_combined',

# Read files - Inpatients & Outpatients Data

In [4]:
# Let's get the inpatient data set
inpatient11 = pd.read_csv("data/inpatient_charges_2011.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
inpatient12 = pd.read_csv("data/inpatient_charges_2012.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
inpatient13 = pd.read_csv("data/inpatient_charges_2013.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
inpatient14 = pd.read_csv("data/inpatient_charges_2014.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

## Transform Inpatient Dataset

In [5]:
# Add year cell to inpatient data set
inpatient11["year"] = "2011"
inpatient12["year"] = "2012"
inpatient13["year"] = "2013"
inpatient14["year"] = "2014"
inpatient14.tail()

,provider_id,provider_name,provider_street_address,provider_city,provider_state,provider_zipcode,drg_definition,hospital_referral_region_description,total_discharges,average_covered_charges,average_total_payments,average_medicare_payments,year
202651,050573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,920 - COMPLICATIONS OF TREATMENT W CC,CA - Palm Springs/Rancho M,21,54910.09524,6994.52381,5516.333333,2014
202652,050573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,948 - SIGNS & SYMPTOMS W/O MCC,CA - Palm Springs/Rancho M,18,57082.77778,4769.444444,3757.666667,2014
202653,050573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,981 - EXTENSIVE O.R. PROCEDURE UNRELATED TO PR...,CA - Palm Springs/Rancho M,23,252399.8261,31364.21739,27667.91304,2014
202654,050573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,982 - EXTENSIVE O.R. PROCEDURE UNRELATED TO PR...,CA - Palm Springs/Rancho M,21,110158.381,17395.95238,16332.80952,2014
202655,050573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,987 - NON-EXTENSIVE O.R. PROC UNRELATED TO PRI...,CA - Palm Springs/Rancho M,11,249276.2727,28050.36364,27168.90909,2014


In [6]:
# Append data frames
append1 = inpatient11.append(inpatient12)
append2 = append1.append(inpatient13)
inpatient_df = append2.append(inpatient14)

inpatient_df.shape

(677068, 13)

In [7]:
inpatient_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 677068 entries, 0 to 202655
Data columns (total 13 columns):
provider_id                             677068 non-null object
provider_name                           677068 non-null object
provider_street_address                 677068 non-null object
provider_city                           677068 non-null object
provider_state                          677068 non-null object
provider_zipcode                        677068 non-null object
drg_definition                          677068 non-null object
hospital_referral_region_description    677068 non-null object
total_discharges                        677068 non-null object
average_covered_charges                 677068 non-null object
average_total_payments                  677068 non-null object
average_medicare_payments               677068 non-null object
year                                    677068 non-null object
dtypes: object(13)
memory usage: 72.3+ MB


In [8]:
# Convert columns to numeric 
# help: https://stackoverflow.com/questions/15891038/change-data-type-of-columns-in-pandas
inpatient_df[["provider_id", "provider_zipcode", "total_discharges", "average_covered_charges", "average_total_payments", "average_medicare_payments", "year"]] = inpatient_df[["provider_id", "provider_zipcode", "total_discharges", "average_covered_charges", "average_total_payments", "average_medicare_payments", "year"]].apply(pd.to_numeric)

In [9]:
print(inpatient_df.info())
inpatient_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 677068 entries, 0 to 202655
Data columns (total 13 columns):
provider_id                             677068 non-null int64
provider_name                           677068 non-null object
provider_street_address                 677068 non-null object
provider_city                           677068 non-null object
provider_state                          677068 non-null object
provider_zipcode                        677068 non-null int64
drg_definition                          677068 non-null object
hospital_referral_region_description    677068 non-null object
total_discharges                        677068 non-null int64
average_covered_charges                 677068 non-null float64
average_total_payments                  677068 non-null float64
average_medicare_payments               677068 non-null float64
year                                    677068 non-null int64
dtypes: float64(3), int64(4), object(6)
memory usage: 72.3+ MB
None


,provider_id,provider_name,provider_street_address,provider_city,provider_state,provider_zipcode,drg_definition,hospital_referral_region_description,total_discharges,average_covered_charges,average_total_payments,average_medicare_payments,year
0,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,AZ - Mesa,12,31522.83333,5780.416667,4845.083333,2011
1,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,AZ - Mesa,33,56245.78788,12109.787880,10928.666670,2011
2,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,AZ - Mesa,50,39587.98000,6890.980000,5716.280000,2011
3,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,AZ - Mesa,26,30465.76923,5521.115385,4080.884615,2011
4,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,069 - TRANSIENT ISCHEMIA,AZ - Mesa,30,34225.53333,4920.133333,3595.933333,2011


In [10]:
# dropping null value columns to avoid errors 
inpatient_df.dropna(inplace = True) 

# Split hospital region and keep city name
# new data frame with split value columns 
new = inpatient_df["hospital_referral_region_description"].str.split("-", n = 1, expand = True) 
  
# making separate first name column from new data frame 
inpatient_df["state"]= new[0] 
  
# making separate last name column from new data frame 
inpatient_df["city"]= new[1] 
  
# Dropping old Name columns 
inpatient_df.drop(columns =["hospital_referral_region_description"], inplace = True) 
inpatient_df.drop(columns =["state"], inplace = True) 

  
# df display 
inpatient_df

,provider_id,provider_name,provider_street_address,provider_city,provider_state,provider_zipcode,drg_definition,total_discharges,average_covered_charges,average_total_payments,average_medicare_payments,year,city
0,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,12,31522.83333,5780.416667,4845.083333,2011,Mesa
1,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,33,56245.78788,12109.787880,10928.666670,2011,Mesa
2,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,50,39587.98000,6890.980000,5716.280000,2011,Mesa
3,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,26,30465.76923,5521.115385,4080.884615,2011,Mesa
4,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,069 - TRANSIENT ISCHEMIA,30,34225.53333,4920.133333,3595.933333,2011,Mesa
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202651,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,920 - COMPLICATIONS OF TREATMENT W CC,21,54910.09524,6994.523810,5516.333333,2014,Palm Springs/Rancho M
202652,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,948 - SIGNS & SYMPTOMS W/O MCC,18,57082.77778,4769.444444,3757.666667,2014,Palm Springs/Rancho M
202653,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,981 - EXTENSIVE O.R. PROCEDURE UNRELATED TO PR...,23,252399.82610,31364.217390,27667.913040,2014,Palm Springs/Rancho M
202654,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,982 - EXTENSIVE O.R. PROCEDURE UNRELATED TO PR...,21,110158.38100,17395.952380,16332.809520,2014,Palm Springs/Rancho M


In [11]:
# Write to CSV file
# inpatient_df.to_csv("data/CLEAN_inpatients_charges_combined.csv", index=False, encoding='utf8')


## Transform Outpatient data set

In [12]:
# Let's get the outpatient data set
outpatient11 = pd.read_csv("data/outpatient_charges_2011.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
outpatient12 = pd.read_csv("data/outpatient_charges_2012.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
outpatient13 = pd.read_csv("data/outpatient_charges_2013.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
outpatient14 = pd.read_csv("data/outpatient_charges_2014.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

In [13]:
# Add year cell to outpatient data set
outpatient11["year"] = "2011"
outpatient12["year"] = "2012"
outpatient13["year"] = "2013"
outpatient14["year"] = "2014"
inpatient14.tail()

,provider_id,provider_name,provider_street_address,provider_city,provider_state,provider_zipcode,drg_definition,hospital_referral_region_description,total_discharges,average_covered_charges,average_total_payments,average_medicare_payments,year
202651,050573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,920 - COMPLICATIONS OF TREATMENT W CC,CA - Palm Springs/Rancho M,21,54910.09524,6994.52381,5516.333333,2014
202652,050573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,948 - SIGNS & SYMPTOMS W/O MCC,CA - Palm Springs/Rancho M,18,57082.77778,4769.444444,3757.666667,2014
202653,050573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,981 - EXTENSIVE O.R. PROCEDURE UNRELATED TO PR...,CA - Palm Springs/Rancho M,23,252399.8261,31364.21739,27667.91304,2014
202654,050573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,982 - EXTENSIVE O.R. PROCEDURE UNRELATED TO PR...,CA - Palm Springs/Rancho M,21,110158.381,17395.95238,16332.80952,2014
202655,050573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,987 - NON-EXTENSIVE O.R. PROC UNRELATED TO PRI...,CA - Palm Springs/Rancho M,11,249276.2727,28050.36364,27168.90909,2014


In [14]:
# Append data frames
out1 = outpatient11.append(outpatient12)
out2 = out1.append(outpatient13)
outpatient_df = out2.append(outpatient14)

outpatient_df.shape

(167405, 12)

In [15]:
print(outpatient_df.info())
outpatient_df.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 167405 entries, 0 to 35894
Data columns (total 12 columns):
provider_id                            167405 non-null object
provider_name                          167405 non-null object
provider_street_address                167405 non-null object
provider_city                          167405 non-null object
provider_state                         167405 non-null object
provider_zipcode                       167405 non-null object
apc                                    167405 non-null object
hospital_referral_region               167405 non-null object
outpatient_services                    167405 non-null object
average_estimated_submitted_charges    167405 non-null object
average_total_payments                 167405 non-null object
year                                   167405 non-null object
dtypes: object(12)
memory usage: 16.6+ MB
None


,provider_id,provider_name,provider_street_address,provider_city,provider_state,provider_zipcode,apc,hospital_referral_region,outpatient_services,average_estimated_submitted_charges,average_total_payments,year
0,030036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0013 - Level II Debridement & Destruction,AZ - Mesa,31,327.5158065,49.88903226,2011
1,030036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0015 - Level III Debridement & Destruction,AZ - Mesa,936,299.8679915,104.4160684,2011
2,030036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0020 - Level II Excision/ Biopsy,AZ - Mesa,14,5409.921429,527.0964286,2011
3,030036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0078 - Level III Pulmonary Treatment,AZ - Mesa,48,566.2758333,101.36,2011
4,030036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0096 - Level II Noninvasive Physiologic Studies,AZ - Mesa,68,767.8220588,107.3352941,2011


In [16]:
# Convert columns to numeric 
# help: https://stackoverflow.com/questions/15891038/change-data-type-of-columns-in-pandas
outpatient_df[["provider_id", "provider_zipcode", "outpatient_services", "average_estimated_submitted_charges", "average_total_payments"]] = outpatient_df[["provider_id", "provider_zipcode", "outpatient_services", "average_estimated_submitted_charges", "average_total_payments"]].apply(pd.to_numeric)
print(outpatient_df.info())
outpatient_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167405 entries, 0 to 35894
Data columns (total 12 columns):
provider_id                            167405 non-null int64
provider_name                          167405 non-null object
provider_street_address                167405 non-null object
provider_city                          167405 non-null object
provider_state                         167405 non-null object
provider_zipcode                       167405 non-null int64
apc                                    167405 non-null object
hospital_referral_region               167405 non-null object
outpatient_services                    167405 non-null int64
average_estimated_submitted_charges    167405 non-null float64
average_total_payments                 167405 non-null float64
year                                   167405 non-null object
dtypes: float64(2), int64(3), object(7)
memory usage: 16.6+ MB
None


,provider_id,provider_name,provider_street_address,provider_city,provider_state,provider_zipcode,apc,hospital_referral_region,outpatient_services,average_estimated_submitted_charges,average_total_payments,year
0,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0013 - Level II Debridement & Destruction,AZ - Mesa,31,327.515806,49.889032,2011
1,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0015 - Level III Debridement & Destruction,AZ - Mesa,936,299.867992,104.416068,2011
2,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0020 - Level II Excision/ Biopsy,AZ - Mesa,14,5409.921429,527.096429,2011
3,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0078 - Level III Pulmonary Treatment,AZ - Mesa,48,566.275833,101.360000,2011
4,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0096 - Level II Noninvasive Physiologic Studies,AZ - Mesa,68,767.822059,107.335294,2011


In [17]:
# dropping null value columns to avoid errors 
inpatient_df.dropna(inplace = True) 
outpatient_df.dropna(inplace = True) 

# Split hospital region and keep city name
# new data frame with split value columns 
new1 = outpatient_df["hospital_referral_region"].str.split("-", n = 1, expand = True) 
  
# making separate first name column from new data frame 
outpatient_df["state"]= new1[0] 
  
# making separate last name column from new data frame 
outpatient_df["city"]= new1[1] 
  
# Dropping old Name columns 
outpatient_df.drop(columns =["hospital_referral_region"], inplace = True) 
outpatient_df.drop(columns =["state"], inplace = True)   
# df display 
outpatient_df


,provider_id,provider_name,provider_street_address,provider_city,provider_state,provider_zipcode,apc,outpatient_services,average_estimated_submitted_charges,average_total_payments,year,city
0,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0013 - Level II Debridement & Destruction,31,327.515806,49.889032,2011,Mesa
1,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0015 - Level III Debridement & Destruction,936,299.867992,104.416068,2011,Mesa
2,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0020 - Level II Excision/ Biopsy,14,5409.921429,527.096429,2011,Mesa
3,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0078 - Level III Pulmonary Treatment,48,566.275833,101.360000,2011,Mesa
4,30036,CHANDLER REGIONAL MEDICAL CENTER,475 SOUTH DOBSON ROAD,CHANDLER,AZ,85224,0096 - Level II Noninvasive Physiologic Studies,68,767.822059,107.335294,2011,Mesa
...,...,...,...,...,...,...,...,...,...,...,...,...
35890,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,0631 - Level 1 Examinations & Related Services,594,93.944411,85.080000,2014,Palm Springs/Rancho M
35891,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,0632 - Level 2 Examinations & Related Services,83,201.224217,108.697952,2014,Palm Springs/Rancho M
35892,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,0634 - Hospital Clinic Visits,173905,226.748456,105.074620,2014,Palm Springs/Rancho M
35893,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,0690 - Level I Electronic Analysis of Devices,7088,105.821893,40.882099,2014,Palm Springs/Rancho M


In [18]:
# # Write to CSV file
# outpatient_df.to_csv("data/CLEAN_outpatients_charges_combined.csv", index=False, encoding='utf8')


# Prepare data for graph

## Prepare inpatient/outpatient data

In [20]:
# See data
inpatient_df.tail()
outpatient_df.tail()

,provider_id,provider_name,provider_street_address,provider_city,provider_state,provider_zipcode,apc,outpatient_services,average_estimated_submitted_charges,average_total_payments,year,city
35890,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,0631 - Level 1 Examinations & Related Services,594,93.944411,85.080000,2014,Palm Springs/Rancho M
35891,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,0632 - Level 2 Examinations & Related Services,83,201.224217,108.697952,2014,Palm Springs/Rancho M
35892,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,0634 - Hospital Clinic Visits,173905,226.748456,105.074620,2014,Palm Springs/Rancho M
35893,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,0690 - Level I Electronic Analysis of Devices,7088,105.821893,40.882099,2014,Palm Springs/Rancho M
35894,50573,EISENHOWER MEDICAL CENTER,39-000 BOB HOPE DRIVE,RANCHO MIRAGE,CA,92270,0692 - Level II Electronic Analysis of Devices,174,332.447299,130.930287,2014,Palm Springs/Rancho M


In [21]:
# Drop unnessary columns
inpatient_df.drop(columns =["provider_street_address"], inplace = True)
inpatient_df.drop(columns =["provider_city"], inplace = True)

# Drop unnessary columns
outpatient_df.drop(columns =["provider_street_address"], inplace = True)
outpatient_df.drop(columns =["provider_city"], inplace = True)


In [22]:
# View data with qgrid
qgrid_inpatient = qgrid.show_grid(inpatient_df, show_toolbar = True)
qgrid_inpatient

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [23]:
# # Write to CSV file
# inpatient_df.to_csv("data/CLEAN_inpatients_charges_combined.csv", index=False, encoding='utf8')

# # Write to CSV file
# outpatient_df.to_csv("data/CLEAN_outpatients_charges_combined.csv", index=False, encoding='utf8')

In [24]:
# Combine one file inpatient and outpatient for metaData
inpatient_df["type"] = "inpatient"
outpatient_df["type"] = "outpatient"


In [25]:
# Merge both data frame
frames = [inpatient_df, outpatient_df]
meta_data_inpatient_outpatient = pd.concat(frames, sort = False)

In [26]:
# See meta_data_shape
meta_data_inpatient_outpatient.shape

(844473, 15)

In [27]:
# Write to CSV file
# meta_data_inpatient_outpatient.to_csv("data/CLEAN_in_out_patients_charges_metadata.csv", index=False, encoding='utf8')


In [28]:
meta_data_inpatient_outpatient.head()

,provider_id,provider_name,provider_state,provider_zipcode,drg_definition,total_discharges,average_covered_charges,average_total_payments,average_medicare_payments,year,city,type,apc,outpatient_services,average_estimated_submitted_charges
0,30036,CHANDLER REGIONAL MEDICAL CENTER,AZ,85224,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,12.0,31522.83333,5780.416667,4845.083333,2011,Mesa,inpatient,NaN,NaN,NaN
1,30036,CHANDLER REGIONAL MEDICAL CENTER,AZ,85224,064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,33.0,56245.78788,12109.787880,10928.666670,2011,Mesa,inpatient,NaN,NaN,NaN
2,30036,CHANDLER REGIONAL MEDICAL CENTER,AZ,85224,065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,50.0,39587.98000,6890.980000,5716.280000,2011,Mesa,inpatient,NaN,NaN,NaN
3,30036,CHANDLER REGIONAL MEDICAL CENTER,AZ,85224,066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,26.0,30465.76923,5521.115385,4080.884615,2011,Mesa,inpatient,NaN,NaN,NaN
4,30036,CHANDLER REGIONAL MEDICAL CENTER,AZ,85224,069 - TRANSIENT ISCHEMIA,30.0,34225.53333,4920.133333,3595.933333,2011,Mesa,inpatient,NaN,NaN,NaN


In [30]:
# # Load to database
# meta_data_inpatient_outpatient.to_sql(name='CLEAN_in_out_patients_charges_metadata', con=engine, if_exists='append', index=False)
# outpatient_df.to_sql(name='CLEAN_outpatients_charges_combined', con=engine, if_exists='append', index=False)
# inpatient_df.to_sql(name='CLEAN_inpatients_charges_combined', con=engine, if_exists='append', index=False)


## More cleaning
### Goal to answer costs questions

In [31]:
# Group data by state and city
inpatient_df_grouped = inpatient_df.groupby(["provider_state", 'city'])
inpatient_df_grouped.head()

,provider_id,provider_name,provider_state,provider_zipcode,drg_definition,total_discharges,average_covered_charges,average_total_payments,average_medicare_payments,year,city,type
0,30036,CHANDLER REGIONAL MEDICAL CENTER,AZ,85224,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,12,31522.83333,5780.416667,4845.083333,2011,Mesa,inpatient
1,30036,CHANDLER REGIONAL MEDICAL CENTER,AZ,85224,064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,33,56245.78788,12109.787880,10928.666670,2011,Mesa,inpatient
2,30036,CHANDLER REGIONAL MEDICAL CENTER,AZ,85224,065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,50,39587.98000,6890.980000,5716.280000,2011,Mesa,inpatient
3,30036,CHANDLER REGIONAL MEDICAL CENTER,AZ,85224,066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,26,30465.76923,5521.115385,4080.884615,2011,Mesa,inpatient
4,30036,CHANDLER REGIONAL MEDICAL CENTER,AZ,85224,069 - TRANSIENT ISCHEMIA,30,34225.53333,4920.133333,3595.933333,2011,Mesa,inpatient
...,...,...,...,...,...,...,...,...,...,...,...,...
69460,370138,MEMORIAL HOSPITAL OF TEXAS COUNTY,OK,73942,195 - SIMPLE PNEUMONIA & PLEURISY W/O CC/MCC,14,11260.85714,5457.142857,3605.785714,2012,Amarillo,inpatient
69461,370138,MEMORIAL HOSPITAL OF TEXAS COUNTY,OK,73942,291 - HEART FAILURE & SHOCK W MCC,13,17970.30769,10974.923080,10049.923080,2012,Amarillo,inpatient
106384,430089,SIOUXLAND SURGERY CENTER LP,SD,57049,460 - SPINAL FUSION EXCEPT CERVICAL W/O MCC,72,130413.11110,25508.888890,23994.819440,2012,Sioux City,inpatient
117297,360261,THREE GABLES SURGERY CENTER,OH,45669,484 - MAJOR JOINT & LIMB REATTACHMENT PROC OF ...,34,27169.73529,11361.117650,10250.882350,2013,Huntington,inpatient


In [51]:
a = inpatient_df_grouped["drg_definition"].value_counts()
a

provider_state  city             drg_definition                                                          
AK               Anchorage       194 - SIMPLE PNEUMONIA & PLEURISY W CC                                      30
                                 392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC                29
                                 470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC    28
                                 871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W MCC                    26
                                 378 - G.I. HEMORRHAGE W CC                                                  25
                                                                                                             ..
WY               Salt Lake City  293 - HEART FAILURE & SHOCK W/O CC/MCC                                       1
                                 310 - CARDIAC ARRHYTHMIA & CONDUCTION DISORDERS W/O CC/MCC                   

In [36]:

inpatient_total_discharges = inpatient_df_grouped["total_discharges"].sum()
inpatient_total_discharges
inpatient_average_charges = inpatient_df_grouped["average_covered_charges"].sum()
inpatient_average_charges

provider_state  city           
AK               Anchorage         5.141184e+07
AL               Birmingham        3.225430e+08
                 Columbus          1.297255e+06
                 Dothan            4.403383e+07
                 Huntsville        6.208354e+07
                                       ...     
WV               Winchester        4.351731e+06
WY               Billings          2.717799e+06
                 Casper            1.507433e+07
                 Fort Collins      1.105501e+07
                 Salt Lake City    2.196810e+06
Name: average_covered_charges, Length: 407, dtype: float64

In [38]:
# Sum numerical data of the inpatients data

inpatient_total_discharges = inpatient_df_grouped["total_discharges"].sum()
inpatient_total_discharges

inpatient_average_charges = inpatient_df_grouped["average_covered_charges"].sum()

inpatient_average_total_payments = inpatient_df_grouped["average_total_payments"].sum()

inpatient_average_medicare_payments = inpatient_df_grouped["average_medicare_payments"].sum()

# Count the number of diagnosis
inpatient_total_MSDRG = inpatient_df_grouped["drg_definition"].count()
inpatient_total_MSDRG

# What is the average cost for inpatient and outpatient treatment in each city and state?

total_average_cost_inpatient = inpatient_average_charges/inpatient_total_MSDRG 
total_average_cost_inpatient = {"avg_charges": inpatient_average_charges,"msdrg_total_count":inpatient_total_MSDRG, "cost_per_treatment":total_average_cost_inpatient}
total_average_cost_inpatient_df = pd.DataFrame(total_average_cost_inpatient)
total_average_cost_inpatient_df = total_average_cost_inpatient_df.reset_index()
total_average_cost_inpatient_df



,provider_state,city,avg_charges,msdrg_total_count,cost_per_treatment
0,AK,Anchorage,5.141184e+07,939,54751.698213
1,AL,Birmingham,3.225430e+08,7453,43276.939887
2,AL,Columbus,1.297255e+06,32,40539.230615
3,AL,Dothan,4.403383e+07,1201,36664.304815
4,AL,Huntsville,6.208354e+07,1614,38465.638893
...,...,...,...,...,...
402,WV,Winchester,4.351731e+06,242,17982.360387
403,WY,Billings,2.717799e+06,137,19837.952076
404,WY,Casper,1.507433e+07,443,34027.838120
405,WY,Fort Collins,1.105501e+07,344,32136.657806


In [43]:
# Merge state name
states = pd.read_csv("data/CLEAN_states_tables.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
states = states.rename(columns = {"state":"provider_state"})
states
total_average_cost_inpatient_df = pd.merge(total_average_cost_inpatient_df,states, on = "provider_state", how = "outer")
total_average_cost_inpatient_df 

,provider_state,city,avg_charges,msdrg_total_count,cost_per_treatment,latitude,longitude,name
0,AK,Anchorage,5.141184e+07,939.0,54751.698213,63.588753000000004,-154.493062,Alaska
1,AL,Birmingham,3.225430e+08,7453.0,43276.939887,32.318231,-86.902298,Alabama
2,AL,Columbus,1.297255e+06,32.0,40539.230615,32.318231,-86.902298,Alabama
3,AL,Dothan,4.403383e+07,1201.0,36664.304815,32.318231,-86.902298,Alabama
4,AL,Huntsville,6.208354e+07,1614.0,38465.638893,32.318231,-86.902298,Alabama
...,...,...,...,...,...,...,...,...
403,WY,Billings,2.717799e+06,137.0,19837.952076,43.075967999999996,-107.29028400000001,Wyoming
404,WY,Casper,1.507433e+07,443.0,34027.838120,43.075967999999996,-107.29028400000001,Wyoming
405,WY,Fort Collins,1.105501e+07,344.0,32136.657806,43.075967999999996,-107.29028400000001,Wyoming
406,WY,Salt Lake City,2.196810e+06,89.0,24683.260283,43.075967999999996,-107.29028400000001,Wyoming


In [44]:
# Export to excel

total_average_cost_inpatient_df.to_csv("data/CLEAN_total_avg_cost_inpatients.csv", index=False, encoding='utf8')

In [35]:
# # Load to database
total_average_cost_inpatient_df.to_sql(name='CLEAN_total_avg_cost_inpatients', con=engine, if_exists='append', index=False)

In [45]:
# Group outpatients data by city and state
outpatient_df_grouped = outpatient_df.groupby(["provider_state", "city"])

# Sum numerical portion of the data
outpatient_services = outpatient_df_grouped["outpatient_services"].sum()
outpatient_average_charges = outpatient_df_grouped["average_estimated_submitted_charges"].sum()
outpatient_average_total_charges = outpatient_df_grouped["average_total_payments"].sum()

# Count the diagnosis
outpatient_total_count_apc = outpatient_df_grouped["apc"].count()
outpatient_total_count_apc

# What is the average cost for inpatient and outpatient treatment in each city and state?
total_average_cost_outpatient = outpatient_average_charges / outpatient_total_count_apc
total_average_cost_outpatient

provider_state  city           
AK               Anchorage         1270.425887
AL               Birmingham        2261.235579
                 Columbus          1687.882790
                 Dothan            1932.883442
                 Huntsville        2469.583766
                                      ...     
WV               Winchester         984.783837
WY               Billings          1211.847287
                 Casper            1817.442408
                 Fort Collins      1429.990101
                 Salt Lake City    1418.854107
Length: 400, dtype: float64

In [46]:
# Convert to data frame export to excel
total_average_cost_outpatient = {"avg_charges":outpatient_average_charges, "apc_total_count":outpatient_total_count_apc, "cost_per_treatment":total_average_cost_outpatient}
total_average_cost_outpatient_df = pd.DataFrame(total_average_cost_outpatient)
total_average_cost_outpatient_df = total_average_cost_outpatient_df.reset_index()
total_average_cost_outpatient_df

,provider_state,city,avg_charges,apc_total_count,cost_per_treatment
0,AK,Anchorage,4.039954e+05,318,1270.425887
1,AL,Birmingham,3.721994e+06,1646,2261.235579
2,AL,Columbus,2.025459e+04,12,1687.882790
3,AL,Dothan,5.759993e+05,298,1932.883442
4,AL,Huntsville,6.791355e+05,275,2469.583766
...,...,...,...,...,...
395,WV,Winchester,6.499573e+04,66,984.783837
396,WY,Billings,1.017952e+05,84,1211.847287
397,WY,Casper,2.090059e+05,115,1817.442408
398,WY,Fort Collins,1.515790e+05,106,1429.990101


In [47]:
# Merge states name
total_average_cost_outpatient_df = pd.merge(total_average_cost_outpatient_df,states, on = "provider_state", how = "outer")
total_average_cost_outpatient_df

,provider_state,city,avg_charges,apc_total_count,cost_per_treatment,latitude,longitude,name
0,AK,Anchorage,4.039954e+05,318.0,1270.425887,63.588753000000004,-154.493062,Alaska
1,AL,Birmingham,3.721994e+06,1646.0,2261.235579,32.318231,-86.902298,Alabama
2,AL,Columbus,2.025459e+04,12.0,1687.882790,32.318231,-86.902298,Alabama
3,AL,Dothan,5.759993e+05,298.0,1932.883442,32.318231,-86.902298,Alabama
4,AL,Huntsville,6.791355e+05,275.0,2469.583766,32.318231,-86.902298,Alabama
...,...,...,...,...,...,...,...,...
397,WY,Casper,2.090059e+05,115.0,1817.442408,43.075967999999996,-107.29028400000001,Wyoming
398,WY,Fort Collins,1.515790e+05,106.0,1429.990101,43.075967999999996,-107.29028400000001,Wyoming
399,WY,Salt Lake City,1.702625e+05,120.0,1418.854107,43.075967999999996,-107.29028400000001,Wyoming
400,MD,NaN,NaN,NaN,NaN,39.045755,-76.641271,Maryland


In [49]:
total_average_cost_outpatient_df.to_csv("data/CLEAN_total_avg_cost_outpatients.csv", index=False, encoding='utf8')
total_average_cost_outpatient_df

,provider_state,city,avg_charges,apc_total_count,cost_per_treatment,latitude,longitude,name
0,AK,Anchorage,4.039954e+05,318.0,1270.425887,63.588753000000004,-154.493062,Alaska
1,AL,Birmingham,3.721994e+06,1646.0,2261.235579,32.318231,-86.902298,Alabama
2,AL,Columbus,2.025459e+04,12.0,1687.882790,32.318231,-86.902298,Alabama
3,AL,Dothan,5.759993e+05,298.0,1932.883442,32.318231,-86.902298,Alabama
4,AL,Huntsville,6.791355e+05,275.0,2469.583766,32.318231,-86.902298,Alabama
...,...,...,...,...,...,...,...,...
397,WY,Casper,2.090059e+05,115.0,1817.442408,43.075967999999996,-107.29028400000001,Wyoming
398,WY,Fort Collins,1.515790e+05,106.0,1429.990101,43.075967999999996,-107.29028400000001,Wyoming
399,WY,Salt Lake City,1.702625e+05,120.0,1418.854107,43.075967999999996,-107.29028400000001,Wyoming
400,MD,NaN,NaN,NaN,NaN,39.045755,-76.641271,Maryland


In [38]:
# # Load to database
total_average_cost_outpatient_df.to_sql(name='CLEAN_total_avg_cost_outpatients', con=engine, if_exists='append', index=False)

### Goal to answer diagnostics questions

In [62]:
# Which are the most common inpatient diagnostic conditions in the United States
# Which city have the most number of cases for each diagnostic condition?
# What are the average payments for these conditions in these cities and how do they compare to the national average?

# Group by state, city and diagnosis

inpatient_df_grouped_drg = inpatient_df.groupby(["provider_state","city","drg_definition"])

# Sum and count specific columns
inpatient_df_grouped_drg_total_payments = inpatient_df_grouped_drg["average_total_payments"].sum()
inpatient_df_grouped_drg_count = inpatient_df_grouped_drg["drg_definition"].count()

# Find average of results
inpatient_df_grouped_drg_avg_payment_condition = inpatient_df_grouped_drg_total_payments / inpatient_df_grouped_drg_count

# Convert to dataframes
inpatient_df_grouped_drg_data = {"drg_count": inpatient_df_grouped_drg_count, "drg_total_payments": inpatient_df_grouped_drg_total_payments, "avg_pay_drg": inpatient_df_grouped_drg_avg_payment_condition }
inpatient_df_grouped_drg_data_df = pd.DataFrame(inpatient_df_grouped_drg_data)
inpatient_df_grouped_drg_data_df = inpatient_df_grouped_drg_data_df.reset_index()
print(inpatient_df_grouped_drg_data_df)




      provider_state             city  \
0                 AK        Anchorage   
1                 AK        Anchorage   
2                 AK        Anchorage   
3                 AK        Anchorage   
4                 AK        Anchorage   
...              ...              ...   
66816             WY   Salt Lake City   
66817             WY   Salt Lake City   
66818             WY   Salt Lake City   
66819             WY   Salt Lake City   
66820             WY   Salt Lake City   

                                          drg_definition  drg_count  \
0      003 - ECMO OR TRACH W MV 96+ HRS OR PDX EXC FA...          1   
1      004 - TRACH W MV 96+ HRS OR PDX EXC FACE, MOUT...          1   
2                     038 - EXTRACRANIAL PROCEDURES W CC          1   
3               039 - EXTRACRANIAL PROCEDURES W/O CC/MCC          5   
4      057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...          3   
...                                                  ...        ...   
66816  641

In [63]:
inpatient_df_grouped_drg_data_df = pd.merge(inpatient_df_grouped_drg_data_df,states, on = "provider_state", how = "outer")
inpatient_df_grouped_drg_data_df

,provider_state,city,drg_definition,drg_count,drg_total_payments,avg_pay_drg,latitude,longitude,name
0,AK,Anchorage,003 - ECMO OR TRACH W MV 96+ HRS OR PDX EXC FA...,1.0,205621.000000,205621.000000,63.588753000000004,-154.493062,Alaska
1,AK,Anchorage,"004 - TRACH W MV 96+ HRS OR PDX EXC FACE, MOUT...",1.0,144232.363600,144232.363600,63.588753000000004,-154.493062,Alaska
2,AK,Anchorage,038 - EXTRACRANIAL PROCEDURES W CC,1.0,13833.750000,13833.750000,63.588753000000004,-154.493062,Alaska
3,AK,Anchorage,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,5.0,44253.467568,8850.693514,63.588753000000004,-154.493062,Alaska
4,AK,Anchorage,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,3.0,36480.102270,12160.034090,63.588753000000004,-154.493062,Alaska
...,...,...,...,...,...,...,...,...,...
66817,WY,Salt Lake City,682 - RENAL FAILURE W MCC,1.0,15992.000000,15992.000000,43.075967999999996,-107.29028400000001,Wyoming
66818,WY,Salt Lake City,683 - RENAL FAILURE W CC,4.0,40164.599643,10041.149911,43.075967999999996,-107.29028400000001,Wyoming
66819,WY,Salt Lake City,690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC,7.0,52065.582307,7437.940330,43.075967999999996,-107.29028400000001,Wyoming
66820,WY,Salt Lake City,871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ H...,3.0,53189.598680,17729.866227,43.075967999999996,-107.29028400000001,Wyoming


In [65]:
#Export to excel
inpatient_df_grouped_drg_data_df.to_csv("data/CLEAN_inpatients_drg_data.csv", index=False, encoding='utf8')

In [66]:
inpatient_df_grouped_drg_data_df

,provider_state,city,drg_definition,drg_count,drg_total_payments,avg_pay_drg,latitude,longitude,name
0,AK,Anchorage,003 - ECMO OR TRACH W MV 96+ HRS OR PDX EXC FA...,1.0,205621.000000,205621.000000,63.588753000000004,-154.493062,Alaska
1,AK,Anchorage,"004 - TRACH W MV 96+ HRS OR PDX EXC FACE, MOUT...",1.0,144232.363600,144232.363600,63.588753000000004,-154.493062,Alaska
2,AK,Anchorage,038 - EXTRACRANIAL PROCEDURES W CC,1.0,13833.750000,13833.750000,63.588753000000004,-154.493062,Alaska
3,AK,Anchorage,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,5.0,44253.467568,8850.693514,63.588753000000004,-154.493062,Alaska
4,AK,Anchorage,057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/...,3.0,36480.102270,12160.034090,63.588753000000004,-154.493062,Alaska
...,...,...,...,...,...,...,...,...,...
66817,WY,Salt Lake City,682 - RENAL FAILURE W MCC,1.0,15992.000000,15992.000000,43.075967999999996,-107.29028400000001,Wyoming
66818,WY,Salt Lake City,683 - RENAL FAILURE W CC,4.0,40164.599643,10041.149911,43.075967999999996,-107.29028400000001,Wyoming
66819,WY,Salt Lake City,690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC,7.0,52065.582307,7437.940330,43.075967999999996,-107.29028400000001,Wyoming
66820,WY,Salt Lake City,871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ H...,3.0,53189.598680,17729.866227,43.075967999999996,-107.29028400000001,Wyoming


In [41]:
# # Export to database 
inpatient_df_grouped_drg_data_df.to_sql(name='CLEAN_inpatients_drg_data', con=engine, if_exists='append', index=False)

In [42]:
# Whch are the most common outpatient diagnostic conditions in the United States
# Which city have the most number of cases for each diagnostic condition?
# What are the average payments for these conditions in these cities and how do they compare to the national average?

# Group by state, city and diagnosis for outpatients
outpatient_df_grouped_apc = outpatient_df.groupby(["provider_state","city","apc"])

# Do calculations
outpatient_df_grouped_apc_total_payments = outpatient_df_grouped_apc["average_total_payments"].sum()
outpatient_df_grouped_apc_count = outpatient_df_grouped_apc["apc"].count()
outpatient_df_grouped_apc_avg_payment_condition = outpatient_df_grouped_apc_total_payments / outpatient_df_grouped_apc_count

# Create data frame
outpatient_df_grouped_apc_data = {"apc_count": outpatient_df_grouped_apc_count, "apc_total_payments": outpatient_df_grouped_apc_total_payments, "avg_pay_apc": outpatient_df_grouped_apc_avg_payment_condition }
outpatient_df_grouped_apc_data_df = pd.DataFrame(outpatient_df_grouped_apc_data)
outpatient_df_grouped_apc_data_df = outpatient_df_grouped_apc_data_df.reset_index()
outpatient_df_grouped_apc_data_df

# Export to excel 
outpatient_df_grouped_apc_data_df.to_csv("data/CLEAN_outpatients_APC_data.csv", index=False, encoding='utf8')


In [43]:
# load to data base
outpatient_df_grouped_apc_data_df.to_sql(name='CLEAN_outpatients_APC_data', con=engine, if_exists='append', index=False)

# Read Files Part D - Data

In [95]:
# Let's get the part-d dataset
part_d_a = pd.read_csv("data/part_d_prescriber_2014a.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
part_d_b = pd.read_csv("data/part_d_prescriber_2014b.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
part_d_c = pd.read_csv("data/part_d_prescriber_2014c.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
part_d_d = pd.read_csv("data/part_d_prescriber_2014d.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

## Questions to answer:
1. What is the total number of medications prescribed in each state?
2. What is the most prescribed medication in each state?

In [96]:
# What is the total number of medications prescribed in each state?
# What is the most prescribed medication in each state?
print(part_d_a.head())
print(part_d_b.head())
print(part_d_c.head())
print(part_d_d.head())

          npi nppes_provider_last_org_name nppes_provider_first_name  \
0  1598870164                          LAM                   PATRICK   
1  1053306662                  BARTHOLOMEW                      DEAN   
2  1023026937                           LU                  LEIGHMIN   
3  1689692592                    STOCKWELL                    PHILIP   
4  1922083104                        CARON                    NORMAN   

  nppes_provider_city nppes_provider_state  
0            HONOLULU                   HI  
1            SARATOGA                   WY  
2           ST THOMAS                   VI  
3     EAST PROVIDENCE                   RI  
4           DUNBARTON                   NH  
          npi                              specialty_description
0  1679570527                                    General Surgery
1  1871535179                                     Anesthesiology
2  1952677445  Student in an Organized Health Care Education/...
3  1780624122                  Osteop

In [100]:
# # Delete unnessary columns
part_d_a = part_d_a.drop(columns =["nppes_provider_last_org_name","nppes_provider_first_name"])
part_d_a
part_d_c = part_d_c.drop(columns =["bene_count"])
part_d_c

,npi,total_claim_count,total_day_supply,total_drug_cost
0,1184696627,54,1740,162.34
1,1104846351,58,3037,14727.27
2,1457376790,62,1818,426.06
3,1326026956,52,1724,842.78
4,1275591026,56,2820,1954.27
...,...,...,...,...
24120613,1770566531,51,1574,12586.71
24120614,1801881214,51,4110,1160.95
24120615,1841285715,51,1504,12694.44
24120616,1851621064,51,4095,24560.45


In [102]:
part_d_a


,npi,nppes_provider_city,nppes_provider_state
0,1598870164,HONOLULU,HI
1,1053306662,SARATOGA,WY
2,1023026937,ST THOMAS,VI
3,1689692592,EAST PROVIDENCE,RI
4,1922083104,DUNBARTON,NH
...,...,...,...
24120613,1326091778,CHARLESTON,WV
24120614,1285687764,WHEELING,WV
24120615,1578686226,PARKERSBURG,WV
24120616,1053370056,HURRICANE,WV


In [101]:
# Export to csv to be safe
part_d_a.to_csv("data/CLEAN_part_d_a.csv", index=False, encoding='utf8')

In [103]:
#This is the dictionary to get the first and second columns all rows to then map it 
part_d_a_dict = {row[0]: row[2] for k, row in part_d_a.iterrows()}
part_d_a_dict

{'1598870164': 'HI',
 '1053306662': 'WY',
 '1023026937': 'VI',
 '1689692592': 'RI',
 '1922083104': 'NH',
 '1265405864': 'SD',
 '1366519563': 'WY',
 '1366507154': 'VT',
 '1750436689': 'NH',
 '1366489585': 'HI',
 '1649325127': 'AK',
 '1164582607': 'DC',
 '1578550265': 'RI',
 '1477565059': 'DC',
 '1255575619': 'ID',
 '1922366244': 'ZZ',
 '1245455286': 'WY',
 '1831201839': 'RI',
 '1215939715': 'NH',
 '1821098633': 'SD',
 '1710264015': 'RI',
 '1568599223': 'VT',
 '1851490999': 'NH',
 '1346249695': 'RI',
 '1407869795': 'NH',
 '1134118748': 'ND',
 '1396771937': 'RI',
 '1679515613': 'VT',
 '1427045301': 'RI',
 '1730168188': 'SD',
 '1760451660': 'SD',
 '1679670939': 'HI',
 '1841223450': 'HI',
 '1588615561': 'NH',
 '1548278146': 'DE',
 '1790758241': 'SD',
 '1992898977': 'RI',
 '1053355669': 'SD',
 '1578547212': 'RI',
 '1841296597': 'RI',
 '1902948516': 'ID',
 '1467416495': 'VT',
 '1548376809': 'WY',
 '1811961139': 'NH',
 '1609872399': 'RI',
 '1790870400': 'ID',
 '1831146877': 'RI',
 '1356397533'

In [105]:
#This is the dictionary to get the first and second columns all rows to then map it 
part_d_a_dict2 = {row[0]: row[1] for k, row in part_d_a.iterrows()}
part_d_a_dict2

{'1598870164': 'HONOLULU',
 '1053306662': 'SARATOGA',
 '1023026937': 'ST THOMAS',
 '1689692592': 'EAST PROVIDENCE',
 '1922083104': 'DUNBARTON',
 '1265405864': 'SPEARFISH',
 '1366519563': 'LANDER',
 '1366507154': 'BARRE',
 '1750436689': 'NEW LONDON',
 '1366489585': 'EWA BEACH',
 '1649325127': 'ANCHORAGE',
 '1164582607': 'WASHINGTON',
 '1578550265': 'EAST PROVIDENCE',
 '1477565059': 'WASHINGTON',
 '1255575619': 'TWIN FALLS',
 '1922366244': 'IRBID',
 '1245455286': 'CHEYENNE',
 '1831201839': 'NORTH PROVIDENCE',
 '1215939715': 'DERRY',
 '1821098633': 'SIOUX FALLS',
 '1710264015': 'WARWICK',
 '1568599223': 'RUTLAND',
 '1851490999': 'HENNIKER',
 '1346249695': 'PROVIDENCE',
 '1407869795': 'NASHUA',
 '1134118748': 'UNDERWOOD',
 '1396771937': 'WAKEFIELD',
 '1679515613': 'NEWBURY',
 '1427045301': 'LINCOLN',
 '1730168188': 'ESTELLINE',
 '1760451660': 'DELL RAPIDS',
 '1679670939': 'HONOLULU',
 '1841223450': 'HONOLULU',
 '1588615561': 'MILFORD',
 '1548278146': 'NEWARK',
 '1790758241': 'SIOUX FALLS',

In [106]:
# Map data state into part_d_d data frame
# part_d_d['state'] = part_d_d['npi'].map(part_d_a_dict)
# part_d_d 
part_d_d['city'] = part_d_d['npi'].map(part_d_a_dict2)
part_d_d

,npi,drug_name,total_claim_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,state,city
0,1013904184,HYDROCHLOROTHIAZIDE,29.0,1852.0,173.93,TN,COOKEVILLE
1,1679576359,HYDROXYUREA,29.0,1482.0,888.73,MO,SAINT LOUIS
2,1730434176,FUROSEMIDE,29.0,1335.0,139.68,WV,HUNTINGTON
3,1174546295,ALENDRONATE SODIUM,29.0,1313.0,294.8,NM,ALBUQUERQUE
4,1194719591,ALLOPURINOL,29.0,1786.0,276.75,NC,NEW BERN
...,...,...,...,...,...,...,...
24120613,1275646952,LEVETIRACETAM,39.0,1126.0,1419.45,CA,AZUSA
24120614,1275734394,OMEPRAZOLE,188.0,10695.0,4077.38,MI,MUSSEY
24120615,1275799819,METOPROLOL TARTRATE,137.0,10800.0,1248.93,NV,PAHRUMP
24120616,1285601153,CARVEDILOL,73.0,2023.0,1664.61,WI,JANESVILLE


In [107]:
# Export to csv just in case
part_d_d.to_csv("data/CLEAN_part_d_d.csv", index=False, encoding='utf8')

In [109]:
# This is the dictionary to map specialty description
part_d_b_dict = {row[0]: row[1] for k, row in part_d_b.iterrows()}
part_d_b_dict


{'1679570527': 'General Surgery',
 '1871535179': 'Anesthesiology',
 '1952677445': 'Student in an Organized Health Care Education/Training Program',
 '1780624122': 'Osteopathic Manipulative Medicine',
 '1811190804': 'Otolaryngology',
 '1427085521': 'Anesthesiology',
 '1457409104': 'Interventional Pain Management',
 '1891051181': 'Student in an Organized Health Care Education/Training Program',
 '1891743241': 'Podiatry',
 '1093711087': 'Podiatry',
 '1568671196': 'Infectious Disease',
 '1538268438': 'Medical Oncology',
 '1972641355': 'Neuropsychiatry',
 '1770650517': 'Infectious Disease',
 '1801199013': 'Optometry',
 '1942403977': 'Psychiatry & Neurology',
 '1881644516': 'Medical Genetics, Ph.D. Medical Genetics',
 '1902249048': 'Student in an Organized Health Care Education/Training Program',
 '1902822166': 'Medical Oncology',
 '1316239833': 'Student in an Organized Health Care Education/Training Program',
 '1295731362': 'Medical Oncology',
 '1396903134': 'Maxillofacial Surgery',
 '15886

In [110]:
# Map both data between part_d_d and part_d_b
part_d_d['specialty_description'] = part_d_d['npi'].map(part_d_b_dict)
part_d_d

,npi,drug_name,total_claim_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,state,city,specialty_description
0,1013904184,HYDROCHLOROTHIAZIDE,29.0,1852.0,173.93,TN,COOKEVILLE,Nurse Practitioner
1,1679576359,HYDROXYUREA,29.0,1482.0,888.73,MO,SAINT LOUIS,Hematology/Oncology
2,1730434176,FUROSEMIDE,29.0,1335.0,139.68,WV,HUNTINGTON,Nurse Practitioner
3,1174546295,ALENDRONATE SODIUM,29.0,1313.0,294.8,NM,ALBUQUERQUE,Family Practice
4,1194719591,ALLOPURINOL,29.0,1786.0,276.75,NC,NEW BERN,Internal Medicine
...,...,...,...,...,...,...,...,...
24120613,1275646952,LEVETIRACETAM,39.0,1126.0,1419.45,CA,AZUSA,General Practice
24120614,1275734394,OMEPRAZOLE,188.0,10695.0,4077.38,MI,MUSSEY,Family Practice
24120615,1275799819,METOPROLOL TARTRATE,137.0,10800.0,1248.93,NV,PAHRUMP,Family Practice
24120616,1285601153,CARVEDILOL,73.0,2023.0,1664.61,WI,JANESVILLE,Family Practice


In [111]:
# Save files to CSV 
part_d_d.to_csv("data/CLEAN_part_d_d.csv", index=False, encoding='utf8')
part_d_d.to_csv("data/CLEAN_part_d_FINAL.csv", index=False, encoding='utf8')
# load to data base
# part_d_d.to_sql(name='data/CLEAN_part_d_FINAL', con=engine, if_exists='append', index=False)

In [3]:
# Read data because kernel restart
part_d_d = pd.read_csv("data/CLEAN_part_d_FINAL.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
part_d_final = part_d_d 
part_d_final

,npi,drug_name,total_claim_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,state,city,specialty_description
0,1013904184,HYDROCHLOROTHIAZIDE,29.0,1852.0,173.93,TN,COOKEVILLE,Nurse Practitioner
1,1679576359,HYDROXYUREA,29.0,1482.0,888.73,MO,SAINT LOUIS,Hematology/Oncology
2,1730434176,FUROSEMIDE,29.0,1335.0,139.68,WV,HUNTINGTON,Nurse Practitioner
3,1174546295,ALENDRONATE SODIUM,29.0,1313.0,294.8,NM,ALBUQUERQUE,Family Practice
4,1194719591,ALLOPURINOL,29.0,1786.0,276.75,NC,NEW BERN,Internal Medicine
...,...,...,...,...,...,...,...,...
24120613,1275646952,LEVETIRACETAM,39.0,1126.0,1419.45,CA,AZUSA,General Practice
24120614,1275734394,OMEPRAZOLE,188.0,10695.0,4077.38,MI,MUSSEY,Family Practice
24120615,1275799819,METOPROLOL TARTRATE,137.0,10800.0,1248.93,NV,PAHRUMP,Family Practice
24120616,1285601153,CARVEDILOL,73.0,2023.0,1664.61,WI,JANESVILLE,Family Practice


In [36]:
# What is the total number of medications prescribed in each state?
# What is the most prescribed medication in each state?

medications_per_state = part_d_final.groupby(["state", "city", "npi"])

medications_per_state.first()


drug_name total_claim_count_ge65  \
state city       npi                                                            
AA    APO        1033255906               TRAMADOL HCL                  152.0   
                 1437240777    CHLORHEXIDINE GLUCONATE                   14.0   
                 1629208145                 PROAIR HFA                   19.0   
                 1659535557           PROMETHAZINE HCL                    0.0   
                 1669645115  HYDROCODONE-ACETAMINOPHEN                    NaN   
...                                                ...                    ...   
ZZ    TORONTO    1437421716                     AVONEX                    0.0   
      TRONDHEIM  1821292087               LEVOFLOXACIN                   13.0   
      WINDSOR    1851594634                LAMOTRIGINE                    0.0   
      WOODBRIDGE 1386081206                 LISINOPRIL                    NaN   
      XXXXXX     1770888786                AMOXICILLIN                    NaN   

                            total_day_supply_ge65 total_drug_cost_ge65  \
state city       npi                                                     
AA    APO        1033255906                1042.0               850.73   
                 1437240777                 264.0                 83.7   
                 1629208145                 138.0               130.46   
                 1659535557                   0.0                  0.0   
                 1669645115                   NaN                  NaN   
...                                           ...                  ...   
ZZ    TORONTO    1437421716                   0.0                  0.0   
      TRONDHEIM  1821292087                  29.0                69.01   
      WINDSOR    1851594634                   0.0                  0.0   
      WOODBRIDGE 1386081206                   NaN                  NaN   
      XXXXXX     1770888786                   NaN                  NaN   

                                    specialty_description  
state city       npi                                       
AA    APO        1033255906  Oral Surgery (dentists only)  
                 1437240777                       Dentist  
                 1629208145               General Surgery  
                 1659535557               Family Practice  
                 1669645115                       Dentist  
...                                                   ...  
ZZ    TORONTO    1437421716                     Neurology  
      TRONDHEIM  1821292087            Emergency Medicine  
      WINDSOR    1851594634           Physician Assistant  
      WOODBRIDGE 1386081206               Family Practice  
      XXXXXX     1770888786                       Dentist  

[837663 rows x 5 columns]

In [16]:
# Get count of drugs per state
medications_per_state_total_prescribed = medications_per_state["drug_name"].count()
medications_per_state_total_prescribed

state  city        npi       
AA     APO         1033255906      5
                   1437240777      2
                   1629208145     13
                   1659535557      4
                   1669645115      2
                                ... 
ZZ     TORONTO     1437421716      7
       TRONDHEIM   1821292087      4
       WINDSOR     1851594634    128
       WOODBRIDGE  1386081206      1
       XXXXXX      1770888786      1
Name: drug_name, Length: 837663, dtype: int64

In [35]:
# Put it in data frame
medications_per_state_total_prescribed = {"total_drug_count": medications_per_state_total_prescribed}
medications_per_state_total_prescribed_df = pd.DataFrame(medications_per_state_total_prescribed)
medications_per_state_total_prescribed_df = medications_per_state_total_prescribed_df.reset_index()
medications_per_state_total_prescribed_df

,index,total_drug_count
0,total_drug_count,state city npi AA APO ...


In [18]:
# Find the drugs most prescribed per state
medications_per_state_most_prescribed = medications_per_state["drug_name"].value_counts()
medications_per_state_most_prescribed

state  city        npi         drug_name                
AA     APO         1033255906  AMOXICILLIN                  1
                               CLINDAMYCIN HCL              1
                               HYDROCODONE-ACETAMINOPHEN    1
                               IBUPROFEN                    1
                               TRAMADOL HCL                 1
                                                           ..
ZZ     WINDSOR     1851594634  XOPENEX HFA                  1
                               ZETIA                        1
                               ZOLPIDEM TARTRATE            1
       WOODBRIDGE  1386081206  LISINOPRIL                   1
       XXXXXX      1770888786  AMOXICILLIN                  1
Name: drug_name, Length: 24029105, dtype: int64

In [34]:
# # Create data frame
medications_per_state_most_prescribed = {"drug_count": medications_per_state_most_prescribed}
medications_per_state_most_prescribed_df = pd.DataFrame(medications_per_state_most_prescribed)
medications_per_state_most_prescribed_df = medications_per_state_most_prescribed_df.reset_index()
medications_per_state_most_prescribed_df

,index,drug_count
0,drug_count,state city npi drug_name ...


In [20]:
# We see that there are garbage in the data
medications_per_state_most_prescribed_df["state"].tolist()

['AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AA',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',
 'AE',

## Web scrapping states and coordinates

In [21]:
# Use Pandas to scrape the table containing states data.
url_states = "https://developers.google.com/public-data/docs/canonical/states_csv"

# Convert table to html
states_table = pd.read_html(url_states)

# Identify what type is the output data
type(states_table)

# Ensure to select the first part of the list
states_table_df = pd.DataFrame(states_table[0])


In [27]:
# states_table_df['location'] = states_table_df['latitude']+ "," +states_table_df['longitude']
# states_table_df

states_table_df['location'] = [', '.join(str(x) for x in y) for y in map(tuple, states_table_df[['latitude', 'longitude']].values)]
states_table_df

,state,latitude,longitude,name,location
0,AK,63.588753,-154.493062,Alaska,"63.588753000000004, -154.493062"
1,AL,32.318231,-86.902298,Alabama,"32.318231, -86.902298"
2,AR,35.201050,-91.831833,Arkansas,"35.20105, -91.83183299999999"
3,AZ,34.048928,-111.093731,Arizona,"34.048928000000004, -111.093731"
4,CA,36.778261,-119.417932,California,"36.778261, -119.41793200000001"
5,CO,39.550051,-105.782067,Colorado,"39.550051, -105.782067"
6,CT,41.603221,-73.087749,Connecticut,"41.603221000000005, -73.087749"
7,DC,38.905985,-77.033418,District of Columbia,"38.905985, -77.03341800000001"
8,DE,38.910832,-75.527670,Delaware,"38.910832, -75.52767"
9,FL,27.664827,-81.515754,Florida,"27.664827000000002, -81.515754"


In [31]:
# states_table_df = states_table_df["location"].astype(float)
# states_table_df.info()

In [32]:
# Save raw data to csv
states_table_df.to_csv("data/CLEAN_states_tables.csv", index=False, encoding='utf8')
# states_table_df.to_sql(name='CLEAN_states_tables', con=engine, if_exists='append', index=False)
states_table_df.head()


,state,latitude,longitude,name,location
0,AK,63.588753,-154.493062,Alaska,"63.588753000000004, -154.493062"
1,AL,32.318231,-86.902298,Alabama,"32.318231, -86.902298"
2,AR,35.201050,-91.831833,Arkansas,"35.20105, -91.83183299999999"
3,AZ,34.048928,-111.093731,Arizona,"34.048928000000004, -111.093731"
4,CA,36.778261,-119.417932,California,"36.778261, -119.41793200000001"


In [33]:
# Merge states and lat long to most precribed drugs data frame

medications_per_state_most_prescribed_df =  pd.merge(medications_per_state_most_prescribed_df,states_table_df, on = "state", how = "outer")
medications_per_state_most_prescribed_df = medications_per_state_most_prescribed_df.dropna()
medications_per_state_most_prescribed_df

,state,city,npi,drug_name,drug_count,latitude,longitude,name,location
287,AK,ADAK,1144644451,DULOXETINE HCL,1,63.588753,-154.493062,Alaska,"63.588753000000004, -154.493062"
288,AK,ANCHOR POINT,1477704187,AMOXICILLIN,1,63.588753,-154.493062,Alaska,"63.588753000000004, -154.493062"
289,AK,ANCHOR POINT,1477704187,CHLORHEXIDINE GLUCONATE,1,63.588753,-154.493062,Alaska,"63.588753000000004, -154.493062"
290,AK,ANCHOR POINT,1477704187,HYDROCODONE-ACETAMINOPHEN,1,63.588753,-154.493062,Alaska,"63.588753000000004, -154.493062"
291,AK,ANCHOR POINT,1477704187,TRAMADOL HCL-ACETAMINOPHEN,1,63.588753,-154.493062,Alaska,"63.588753000000004, -154.493062"
...,...,...,...,...,...,...,...,...,...
24028084,WY,WORLAND,1952333221,RANITIDINE HCL,1,43.075968,-107.290284,Wyoming,"43.075967999999996, -107.29028400000001"
24028085,WY,WORLAND,1952333221,SIMVASTATIN,1,43.075968,-107.290284,Wyoming,"43.075967999999996, -107.29028400000001"
24028086,WY,WORLAND,1952333221,SUCRALFATE,1,43.075968,-107.290284,Wyoming,"43.075967999999996, -107.29028400000001"
24028087,WY,WORLAND,1952333221,SULFAMETHOXAZOLE-TRIMETHOPRIM,1,43.075968,-107.290284,Wyoming,"43.075967999999996, -107.29028400000001"


In [124]:
medications_per_state_most_prescribed_df

,state,city,drug_name,drug_count,latitude,longitude,name
188,AK,ADAK,DULOXETINE HCL,1,63.588753,-154.493062,Alaska
189,AK,ANCHOR POINT,AMOXICILLIN,1,63.588753,-154.493062,Alaska
190,AK,ANCHOR POINT,CHLORHEXIDINE GLUCONATE,1,63.588753,-154.493062,Alaska
191,AK,ANCHOR POINT,HYDROCODONE-ACETAMINOPHEN,1,63.588753,-154.493062,Alaska
192,AK,ANCHOR POINT,TRAMADOL HCL-ACETAMINOPHEN,1,63.588753,-154.493062,Alaska
...,...,...,...,...,...,...,...
3547705,WY,WORLAND,VOLTAREN,1,43.075968,-107.290284,Wyoming
3547706,WY,WORLAND,VYTORIN,1,43.075968,-107.290284,Wyoming
3547707,WY,WORLAND,XARELTO,1,43.075968,-107.290284,Wyoming
3547708,WY,WORLAND,XOPENEX HFA,1,43.075968,-107.290284,Wyoming


In [125]:
# Reset index
medications_per_state_most_prescribed_df = medications_per_state_most_prescribed_df.reset_index(drop = True)
medications_per_state_most_prescribed_df

,state,city,drug_name,drug_count,latitude,longitude,name
0,AK,ADAK,DULOXETINE HCL,1,63.588753,-154.493062,Alaska
1,AK,ANCHOR POINT,AMOXICILLIN,1,63.588753,-154.493062,Alaska
2,AK,ANCHOR POINT,CHLORHEXIDINE GLUCONATE,1,63.588753,-154.493062,Alaska
3,AK,ANCHOR POINT,HYDROCODONE-ACETAMINOPHEN,1,63.588753,-154.493062,Alaska
4,AK,ANCHOR POINT,TRAMADOL HCL-ACETAMINOPHEN,1,63.588753,-154.493062,Alaska
...,...,...,...,...,...,...,...
3545257,WY,WORLAND,VOLTAREN,1,43.075968,-107.290284,Wyoming
3545258,WY,WORLAND,VYTORIN,1,43.075968,-107.290284,Wyoming
3545259,WY,WORLAND,XARELTO,1,43.075968,-107.290284,Wyoming
3545260,WY,WORLAND,XOPENEX HFA,1,43.075968,-107.290284,Wyoming


## What is the drug most consumed per state?

In [126]:
# Sort values by drug count in a way that we keep the highest value per state
medications_per_state_most_prescribed_df_data =medications_per_state_most_prescribed_df.sort_values(['drug_count'], ascending=[False]).drop_duplicates(['state']).reset_index(drop=True)
medications_per_state_most_prescribed_df_data

,state,city,drug_name,drug_count,latitude,longitude,name
0,NY,NEW YORK,AMLODIPINE BESYLATE,3120,43.299428,-74.217933,New York
1,TX,HOUSTON,HYDROCODONE-ACETAMINOPHEN,3062,31.968599,-99.901813,Texas
2,IL,CHICAGO,HYDROCODONE-ACETAMINOPHEN,2848,40.633125,-89.398528,Illinois
3,CA,LOS ANGELES,HYDROCODONE-ACETAMINOPHEN,2272,36.778261,-119.417932,California
4,PA,PHILADELPHIA,LISINOPRIL,1961,41.203322,-77.194525,Pennsylvania
5,IN,INDIANAPOLIS,HYDROCODONE-ACETAMINOPHEN,1873,40.551217,-85.602364,Indiana
6,MA,BOSTON,OMEPRAZOLE,1618,42.407211,-71.382437,Massachusetts
7,OR,PORTLAND,HYDROCODONE-ACETAMINOPHEN,1518,43.804133,-120.554201,Oregon
8,NV,LAS VEGAS,HYDROCODONE-ACETAMINOPHEN,1510,38.802610,-116.419389,Nevada
9,AZ,PHOENIX,HYDROCODONE-ACETAMINOPHEN,1483,34.048928,-111.093731,Arizona


In [128]:
# Save to CSV and push to sql
medications_per_state_most_prescribed_df_data.to_csv("data/CLEAN_medications_per_state_most_prescribed_df_data.csv", index=False, encoding='utf8')
# medications_per_state_most_prescribed_df_data.to_sql(name='CLEAN_medications_per_state_most_prescribed_df_data', con=engine, if_exists='append', index=False)

## What is the total number of drugs per state?

In [129]:
# This data is to merge lat long to the data frame of total drug count per state
medications_per_state_total_prescribed_df_data = pd.merge(medications_per_state_total_prescribed_df,states_table_df, on = "state", how = "outer")
medications_per_state_total_prescribed_df_data = medications_per_state_total_prescribed_df_data.dropna()


In [130]:
# Sorting values by name to make sure there are no repetitive states
medications_per_state_total_prescribed_df_data = medications_per_state_total_prescribed_df_data.sort_values("name", ascending = True).reset_index(drop = True)
medications_per_state_total_prescribed_df_data

,state,city,total_drug_count,latitude,longitude,name
0,AL,NEWTON,257,32.318231,-86.902298,Alabama
1,AL,HEFLIN,484,32.318231,-86.902298,Alabama
2,AL,HEADLAND,310,32.318231,-86.902298,Alabama
3,AL,HAZEL GREEN,356,32.318231,-86.902298,Alabama
4,AL,HAYNEVILLE,199,32.318231,-86.902298,Alabama
...,...,...,...,...,...,...
16722,WY,HANNA,35,43.075968,-107.290284,Wyoming
16723,WY,JACKSON,863,43.075968,-107.290284,Wyoming
16724,WY,KEMMERER,285,43.075968,-107.290284,Wyoming
16725,WY,LARAMIE,1156,43.075968,-107.290284,Wyoming


In [131]:
medications_per_state_total_prescribed_df_data.to_csv("data/CLEAN_medications_per_state_total_prescribed_df_data.csv", index=False, encoding='utf8')
# medications_per_state_total_prescribed_df_data.to_sql(name='CLEAN_medications_per_state_total_prescribed_df_data', con=engine, if_exists='append', index=False)

## Most expensive drug per state?

In [132]:
# Take a loow at the raw data

part_d_final.head()

,npi,drug_name,total_claim_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,state,city,specialty_description
0,1013904184,HYDROCHLOROTHIAZIDE,29.0,1852.0,173.93,TN,COOKEVILLE,Nurse Practitioner
1,1679576359,HYDROXYUREA,29.0,1482.0,888.73,MO,SAINT LOUIS,Hematology/Oncology
2,1730434176,FUROSEMIDE,29.0,1335.0,139.68,WV,HUNTINGTON,Nurse Practitioner
3,1174546295,ALENDRONATE SODIUM,29.0,1313.0,294.8,NM,ALBUQUERQUE,Family Practice
4,1194719591,ALLOPURINOL,29.0,1786.0,276.75,NC,NEW BERN,Internal Medicine


In [133]:
part_d_final["total_claim_count_ge65"]= part_d_final["total_claim_count_ge65"].astype(float)
part_d_final["total_day_supply_ge65"] = part_d_final["total_day_supply_ge65"].astype(float)
part_d_final["total_drug_cost_ge65"] = part_d_final["total_drug_cost_ge65"].astype(float)

In [134]:
part_d_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24120618 entries, 0 to 24120617
Data columns (total 8 columns):
npi                       object
drug_name                 object
total_claim_count_ge65    float64
total_day_supply_ge65     float64
total_drug_cost_ge65      float64
state                     object
city                      object
specialty_description     object
dtypes: float64(3), object(5)
memory usage: 1.4+ GB


In [135]:
medications_per_state_cost = part_d_final.groupby(["state","city","drug_name","specialty_description"])
medications_per_state_cost.head()

,npi,drug_name,total_claim_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,state,city,specialty_description
0,1013904184,HYDROCHLOROTHIAZIDE,29.0,1852.0,173.93,TN,COOKEVILLE,Nurse Practitioner
1,1679576359,HYDROXYUREA,29.0,1482.0,888.73,MO,SAINT LOUIS,Hematology/Oncology
2,1730434176,FUROSEMIDE,29.0,1335.0,139.68,WV,HUNTINGTON,Nurse Practitioner
3,1174546295,ALENDRONATE SODIUM,29.0,1313.0,294.80,NM,ALBUQUERQUE,Family Practice
4,1194719591,ALLOPURINOL,29.0,1786.0,276.75,NC,NEW BERN,Internal Medicine
...,...,...,...,...,...,...,...,...
24120603,1245239102,PANTOPRAZOLE SODIUM,38.0,1470.0,318.35,AL,VALLEY,Emergency Medicine
24120609,1255374302,LEVETIRACETAM,53.0,2070.0,1588.21,TX,CARROLLTON,Neurology
24120611,1275538019,SERTRALINE HCL,75.0,3435.0,832.37,TX,GLADEWATER,Family Practice
24120613,1275646952,LEVETIRACETAM,39.0,1126.0,1419.45,CA,AZUSA,General Practice


In [136]:
medications_per_state_cost_claims = medications_per_state_cost["total_claim_count_ge65"].sum()

In [137]:
medications_per_state_cost_suppply = medications_per_state_cost["total_day_supply_ge65"].sum()

In [138]:
medications_per_state_cost_cost = medications_per_state_cost["total_drug_cost_ge65"].sum()

In [139]:
medications_per_state_cost_claims

state  city        drug_name                      specialty_description       
AA     APO         AMOX TR-POTASSIUM CLAVULANATE  General Surgery                  15.0
                   AMOXICILLIN                    Dentist                          16.0
                                                  Oral Surgery (dentists only)    152.0
                   AZITHROMYCIN                   General Surgery                  51.0
                   CEFDINIR                       General Surgery                   0.0
                                                                                  ...  
ZZ     WINDSOR     XOPENEX HFA                    Physician Assistant               0.0
                   ZETIA                          Physician Assistant               0.0
                   ZOLPIDEM TARTRATE              Physician Assistant              18.0
       WOODBRIDGE  LISINOPRIL                     Family Practice                   0.0
       XXXXXX      AMOXICILLIN           

In [144]:
medication_per_state_cost_data = {"drug_claim_count": medications_per_state_cost_claims, "drug_supply_count":medications_per_state_cost_suppply, "drug_cost": medications_per_state_cost_cost }
medication_per_state_cost_data_df = pd.DataFrame(medication_per_state_cost_data)


In [145]:
medication_per_state_cost_data_df = medication_per_state_cost_data_df.reset_index()
medication_per_state_cost_data_df 

,state,city,drug_name,specialty_description,drug_claim_count,drug_supply_count,drug_cost
0,AA,APO,AMOX TR-POTASSIUM CLAVULANATE,General Surgery,15.0,150.0,231.32
1,AA,APO,AMOXICILLIN,Dentist,16.0,88.0,61.78
2,AA,APO,AMOXICILLIN,Oral Surgery (dentists only),152.0,1042.0,850.73
3,AA,APO,AZITHROMYCIN,General Surgery,51.0,251.0,344.10
4,AA,APO,CEFDINIR,General Surgery,0.0,0.0,0.00
...,...,...,...,...,...,...,...
8193718,ZZ,WINDSOR,XOPENEX HFA,Physician Assistant,0.0,0.0,0.00
8193719,ZZ,WINDSOR,ZETIA,Physician Assistant,0.0,0.0,0.00
8193720,ZZ,WINDSOR,ZOLPIDEM TARTRATE,Physician Assistant,18.0,540.0,162.26
8193721,ZZ,WOODBRIDGE,LISINOPRIL,Family Practice,0.0,0.0,0.00


In [146]:
# Sorting values by drug cost to make sure there are no repetitive states
medication_per_state_cost_data_df = medication_per_state_cost_data_df.sort_values(['drug_cost'], ascending=[False]).drop_duplicates(['city']).reset_index(drop=True)
medication_per_state_cost_data_df

,state,city,drug_name,specialty_description,drug_claim_count,drug_supply_count,drug_cost
0,NY,BROOKLYN,NEXIUM,Internal Medicine,72202.0,2303192.0,18969703.78
1,NY,NEW YORK,SOVALDI,Gastroenterology,603.0,16886.0,17236901.45
2,CA,LOS ANGELES,SOVALDI,Gastroenterology,513.0,13960.0,14194735.30
3,TX,HOUSTON,SOVALDI,Gastroenterology,455.0,12712.0,12888956.20
4,MO,SAINT LOUIS,SOVALDI,Gastroenterology,420.0,11760.0,11924902.80
...,...,...,...,...,...,...,...
12143,ME,BOWDOINHAM,HYDROCODONE-ACETAMINOPHEN,Osteopathic Manipulative Medicine,0.0,0.0,0.00
12144,ME,FT FAIRFIELD,AMOXICILLIN,Dentist,0.0,0.0,0.00
12145,ME,EAST WILTON,AMOXICILLIN,Dentist,0.0,0.0,0.00
12146,ME,E WATERBORO,TIZANIDINE HCL,Pediatric Medicine,0.0,0.0,0.00


In [147]:
medication_per_state_cost_data_df.to_csv("data/fix.csv", index=False, encoding='utf8')

In [148]:
medication_per_state_cost_data_df_clean = pd.read_csv("data/fix_clean.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
medication_per_state_cost_data_df_clean

,state,city,drug_name,specialty_description,drug_claim_count,drug_supply_count,drug_cost
0,AK,ANCHORAGE,SOVALDI,Gastroenterology,35,980,1003035.57
1,AK,FAIRBANKS,XYREM,Physician Assistant,12,360,118057.41
2,AK,WASILLA,NEXIUM,Internal Medicine,543,11227,108272.22
3,AK,KETCHIKAN,REVLIMID,Interventional Pain Management,17,252,107561.86
4,AK,SOLDOTNA,XYREM,Family Practice,11,330,90846.88
...,...,...,...,...,...,...,...
12077,WY,GREYBULL,LATANOPROST,Optometry,27,875,379.56
12078,WY,MORAN,SIMVASTATIN,Physician Assistant,11,330,37.4
12079,WY,BIG PINEY,HYDROCODONE-ACETAMINOPHEN,Dentist,0,0,0
12080,WY,RANCHESTER,CLINDAMYCIN HCL,Dentist,0,0,0


In [149]:
medication_per_state_cost_data_df_clean= pd.merge(medication_per_state_cost_data_df_clean,states_table_df, on = "state", how = "outer")
medication_per_state_cost_data_df_clean.head()


,state,city,drug_name,specialty_description,drug_claim_count,drug_supply_count,drug_cost,latitude,longitude,name
0,AK,ANCHORAGE,SOVALDI,Gastroenterology,35,980,1003035.57,63.588753,-154.493062,Alaska
1,AK,FAIRBANKS,XYREM,Physician Assistant,12,360,118057.41,63.588753,-154.493062,Alaska
2,AK,WASILLA,NEXIUM,Internal Medicine,543,11227,108272.22,63.588753,-154.493062,Alaska
3,AK,KETCHIKAN,REVLIMID,Interventional Pain Management,17,252,107561.86,63.588753,-154.493062,Alaska
4,AK,SOLDOTNA,XYREM,Family Practice,11,330,90846.88,63.588753,-154.493062,Alaska


In [150]:
medication_per_state_cost_data_df_clean = medication_per_state_cost_data_df_clean.dropna()
medication_per_state_cost_data_df_clean

,state,city,drug_name,specialty_description,drug_claim_count,drug_supply_count,drug_cost,latitude,longitude,name
0,AK,ANCHORAGE,SOVALDI,Gastroenterology,35,980,1003035.57,63.588753,-154.493062,Alaska
1,AK,FAIRBANKS,XYREM,Physician Assistant,12,360,118057.41,63.588753,-154.493062,Alaska
2,AK,WASILLA,NEXIUM,Internal Medicine,543,11227,108272.22,63.588753,-154.493062,Alaska
3,AK,KETCHIKAN,REVLIMID,Interventional Pain Management,17,252,107561.86,63.588753,-154.493062,Alaska
4,AK,SOLDOTNA,XYREM,Family Practice,11,330,90846.88,63.588753,-154.493062,Alaska
...,...,...,...,...,...,...,...,...,...,...
12077,WY,GREYBULL,LATANOPROST,Optometry,27,875,379.56,43.075968,-107.290284,Wyoming
12078,WY,MORAN,SIMVASTATIN,Physician Assistant,11,330,37.4,43.075968,-107.290284,Wyoming
12079,WY,BIG PINEY,HYDROCODONE-ACETAMINOPHEN,Dentist,0,0,0,43.075968,-107.290284,Wyoming
12080,WY,RANCHESTER,CLINDAMYCIN HCL,Dentist,0,0,0,43.075968,-107.290284,Wyoming


In [152]:
medication_per_state_cost_data_df_clean.to_csv("data/CLEAN_medication_per_state_cost_data.csv", index=False, encoding='utf8')
# medication_per_state_cost_data_df_clean.to_sql(name='CLEAN_medication_per_state_cost_data', con=engine, if_exists='append', index=False)

## Merge all three dataframes of the questions together

In [153]:
medications_per_state_most_prescribed_df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 7 columns):
state         52 non-null object
city          52 non-null object
drug_name     52 non-null object
drug_count    52 non-null int64
latitude      52 non-null float64
longitude     52 non-null float64
name          52 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 3.0+ KB


In [154]:
medications_per_state_total_prescribed_df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16727 entries, 0 to 16726
Data columns (total 6 columns):
state               16727 non-null object
city                16727 non-null object
total_drug_count    16727 non-null int64
latitude            16727 non-null float64
longitude           16727 non-null float64
name                16727 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 784.2+ KB


In [155]:
medication_per_state_cost_data_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12050 entries, 0 to 12081
Data columns (total 10 columns):
state                    12050 non-null object
city                     12050 non-null object
drug_name                12050 non-null object
specialty_description    12050 non-null object
drug_claim_count         12050 non-null object
drug_supply_count        12050 non-null object
drug_cost                12050 non-null object
latitude                 12050 non-null float64
longitude                12050 non-null float64
name                     12050 non-null object
dtypes: float64(2), object(8)
memory usage: 1.0+ MB


In [156]:
merge1= pd.merge(medications_per_state_most_prescribed_df_data,medications_per_state_total_prescribed_df_data, on = "state", how = "inner")
merge1

,state,city_x,drug_name,drug_count,latitude_x,longitude_x,name_x,city_y,total_drug_count,latitude_y,longitude_y,name_y
0,NY,NEW YORK,AMLODIPINE BESYLATE,3120,43.299428,-74.217933,New York,NEW YORK,218700,43.299428,-74.217933,New York
1,NY,NEW YORK,AMLODIPINE BESYLATE,3120,43.299428,-74.217933,New York,CENEREACH,2,43.299428,-74.217933,New York
2,NY,NEW YORK,AMLODIPINE BESYLATE,3120,43.299428,-74.217933,New York,CENTER MORICHES,721,43.299428,-74.217933,New York
3,NY,NEW YORK,AMLODIPINE BESYLATE,3120,43.299428,-74.217933,New York,CENTEREACH,733,43.299428,-74.217933,New York
4,NY,NEW YORK,AMLODIPINE BESYLATE,3120,43.299428,-74.217933,New York,CENTERPORT,43,43.299428,-74.217933,New York
...,...,...,...,...,...,...,...,...,...,...,...,...
16722,VT,BURLINGTON,GABAPENTIN,101,44.558803,-72.577841,Vermont,SHARON,2,44.558803,-72.577841,Vermont
16723,VT,BURLINGTON,GABAPENTIN,101,44.558803,-72.577841,Vermont,SAINT JOHNSBURY,21,44.558803,-72.577841,Vermont
16724,VT,BURLINGTON,GABAPENTIN,101,44.558803,-72.577841,Vermont,SAINT ALBANS,1228,44.558803,-72.577841,Vermont
16725,VT,BURLINGTON,GABAPENTIN,101,44.558803,-72.577841,Vermont,S BURLINGTON,47,44.558803,-72.577841,Vermont


In [157]:
part_d_Data_df_clean = pd.merge(merge1,medication_per_state_cost_data_df_clean, on = "state", how = "outer")

In [158]:
part_d_Data_df_clean

,state,city_x,drug_name_x,drug_count,latitude_x,longitude_x,name_x,city_y,total_drug_count,latitude_y,...,name_y,city,drug_name_y,specialty_description,drug_claim_count,drug_supply_count,drug_cost,latitude,longitude,name
0,NY,NEW YORK,AMLODIPINE BESYLATE,3120,43.299428,-74.217933,New York,NEW YORK,218700,43.299428,...,New York,BROOKLYN,NEXIUM,Internal Medicine,72202,2303192,18969703.78,43.299428,-74.217933,New York
1,NY,NEW YORK,AMLODIPINE BESYLATE,3120,43.299428,-74.217933,New York,NEW YORK,218700,43.299428,...,New York,NEW YORK,SOVALDI,Gastroenterology,603,16886,17236901.45,43.299428,-74.217933,New York
2,NY,NEW YORK,AMLODIPINE BESYLATE,3120,43.299428,-74.217933,New York,NEW YORK,218700,43.299428,...,New York,BRONX,JANUVIA,Internal Medicine,23769,805748,7935205.28,43.299428,-74.217933,New York
3,NY,NEW YORK,AMLODIPINE BESYLATE,3120,43.299428,-74.217933,New York,NEW YORK,218700,43.299428,...,New York,FLUSHING,SOVALDI,Gastroenterology,191,5348,5468421.55,43.299428,-74.217933,New York
4,NY,NEW YORK,AMLODIPINE BESYLATE,3120,43.299428,-74.217933,New York,NEW YORK,218700,43.299428,...,New York,VALHALLA,SOVALDI,Gastroenterology,157,4396,4480539.62,43.299428,-74.217933,New York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6642298,VT,BURLINGTON,GABAPENTIN,101,44.558803,-72.577841,Vermont,RUTLAND,4719,44.558803,...,Vermont,FLORLAM PARK,SIMVASTATIN,Urology,12,360,54.72,44.558803,-72.577841,Vermont
6642299,VT,BURLINGTON,GABAPENTIN,101,44.558803,-72.577841,Vermont,RUTLAND,4719,44.558803,...,Vermont,SOUTH WOODSTOCK,TRAZODONE HCL,Internal Medicine,0,0,0,44.558803,-72.577841,Vermont
6642300,VT,BURLINGTON,GABAPENTIN,101,44.558803,-72.577841,Vermont,RUTLAND,4719,44.558803,...,Vermont,ST ALBANS V,ESTRADIOL,Obstetrics/Gynecology,0,0,0,44.558803,-72.577841,Vermont
6642301,VT,BURLINGTON,GABAPENTIN,101,44.558803,-72.577841,Vermont,RUTLAND,4719,44.558803,...,Vermont,UNDERHILL,SUBOXONE,General Practice,0,0,0,44.558803,-72.577841,Vermont


In [65]:
part_d_Data_df_clean.drop(columns =["latitude_x","longitude_x","latitude_y","longitude_y","name_y","drug_name_y"], inplace = True)


In [66]:
part_d_Data_df_clean = part_d_Data_df_clean.rename(columns = {"drug_name_x":"drug_name", "drug_count":"drug_count_most_consumed"})
part_d_Data_df_clean

,state,drug_name,drug_count_most_consumed,total_drug_count,specialty_description,drug_claim_count,drug_supply_count,drug_cost,avg_cost_drug
0,CA,HYDROCODONE-ACETAMINOPHEN,40167,2314491,Pharmacist,12,360,9999,833
1,TX,HYDROCODONE-ACETAMINOPHEN,24697,1472194,Cardiology,23,2070,999,43
2,NY,AMLODIPINE BESYLATE,18992,1650337,Orthopedic Surgery,20,600,10000,500
3,FL,LISINOPRIL,17115,1611230,Family Medicine,27,1050,10000,370
4,MI,HYDROCODONE-ACETAMINOPHEN,15650,824210,Multispecialty Clinic/Group Practice,32,1680,9997,312
5,IL,HYDROCODONE-ACETAMINOPHEN,14241,866555,Internal Medicine,11,308,10000,909
6,OH,HYDROCODONE-ACETAMINOPHEN,13404,1001853,Internal Medicine,12,360,9993,833
7,PA,LISINOPRIL,13276,1229100,Case Manager/Care Coordinator,242,5148,10000,41
8,NC,HYDROCODONE-ACETAMINOPHEN,11982,876296,Rheumatology,49,1690,9997,204
9,TN,HYDROCODONE-ACETAMINOPHEN,10345,634761,Multispecialty Clinic/Group Practice,12,336,999,83


In [67]:
part_d_Data_df_clean.to_csv("data/CLEAN_part_d_Data.csv", index=False, encoding='utf8')
part_d_Data_df_clean.to_sql(name='CLEAN_part_d_Data', con=engine, if_exists='append', index=False)